[색인 & 가중치 => 학습]
1. 수집한 문서 목록 가져오기
2. 각 문서에서 색인어 목록 추출 전처리 -> 어절 -> 형태소+명사+바이그램
3. 가중치 계산(TF-IDF) -> TDM -> TWM(with DocumentVectorLength)

[질의 -> 색인과정의 2 to 3]
4. 질의에서 색인어 추출
5. 가중치 계산
6. 유사도 계산(코싸인)
7. 유사도 순 정렬(거리-오름차순, 각도-내림차순)
8. 검색 결과 출력


## 1. 모든 문서들에 대해 토큰화 수행하기

In [1]:
# 이 코드에서는 지금까지 배운 내용들을 활용해 텍스트 문서에서 쿼리 단어를 검색한 후 해당 쿼리가 위치한 문서 경로를 출력해 봅니다. 

# 파이썬 버전확인
import sys
print("--sys.version--")
print(sys.version)

print("\n--sys.version_info--")
print(sys.version_info)

print("\n--sys.hexversion--")
print(sys.hexversion)


def getFileList(base = "./", ext = "txt"): # 아무것도 안했다면 base는 현재 경로
    fileList = list()
    for file in listdir(base):
        if file.split(".")[-1] == ext: # .을 기준으로 split한 것이 txt인지 검사
            fileList.append("{0}/{1}".format(base, file))
            
    return fileList

# txt 컨텐츠를 편하게 읽어오기 위한 함수
def getContent(file): 
    with open(file, encoding="UTF-8") as f:
        content = f.read()
    return content

--sys.version--
3.7.1 (default, Dec 10 2018, 22:54:23) [MSC v.1915 64 bit (AMD64)]

--sys.version_info--
sys.version_info(major=3, minor=7, micro=1, releaselevel='final', serial=0)

--sys.hexversion--
50790896


In [2]:
def ngramEojeol(sentence, n=2): # sentence를 받아 어절 단위로 분리해주는 함수
    '''
    입력:     단어1,   단어2,   단어3,  단어4 : 4
    출력(2) : 단어12,  단어23,  단어34 :        3 - n + 1
    출력(3) : 단어123, 단어234         :        2 - n + 1
    '''
    tokens = sentence.split()
    ngram = []
    
    for i in range(len(tokens) - n + 1):
        ngram.append(' '.join(tokens[i:i + n]))    
        
    return ngram

In [3]:
def ngramUmjeol(term, n = 2): # 음절 단위로 구분하는 함수. sentence를 받아 2개(n=2)씩 쪼갠다.

    ngram = []
    
    for i in range(len(term) - n + 1):
        # ngram.append(tokens_ngram[i:i+n]) # 방법1
        # ngram.append(tuple(tokens_ngram[i:i+n])) # 방법2 (튜플로 반환 시 키값을 쓸 수 있음)
        ngram.append(''.join(term[i:i + n])) # 방법3
        
    return ngram

In [4]:
import re
from string import punctuation


def getPatternList():
    patternList = dict()
    pattern = re.compile(r"[%s]{1,}" % re.escape(punctuation)) # punctuation 안의 특수문자가 두번이상 반복되는 모든 문자에 대해 패턴 정의
    patternList["Punctuation"] = pattern
    pattern2 = re.compile(r"([A-Za-z0-9\-\_\.]{3,}@[A-Za-z0-9\-\_]{3,}(.[A-Za-z]{2,})+)") #이메일주소제거패턴
    patternList["Email"] = pattern2
    pattern3 = re.compile(r"([A-Za-z0-9\-\_]{1,}(.[A-Za-z]{2,})+)") # 신문사도메인패턴
    patternList["Domain"] = pattern3
    pattern4 = re.compile(r"\s{2,}") # 두칸짜리 공백제거
    patternList["Whitespace"] = pattern4
    pattern5 = re.compile(r"([^ㄱ-ㅎㅏ-ㅣ가-힣0-9]+)") # 한글이 아닌 영어 기호 제거
    patternList["Korean"] = pattern5
    pattern6 = re.compile(r'[A-Za-z-_]{8,}') # 영어 (대소문자) +\-\_ 제거 (8글자 이상)
    patternList["ElimLongEng"] = pattern6
    pattern7 = re.compile(r"를 2019 및 금지|네이버 채널에서|저작권자 주 블로터앤미디어 저작권자를 명기하고 내용을 변경하지 않으며 비상업적으로 이용하는 조건아래 가능합니다|"
                          r"디지털 마케팅의 미래 아이포럼|무단전재|오류를 우회하기 위한 함수 추가|재배포|기자|아이뉴스24|구독해주세요|"
                          r"고수들의 재테크 비법|이건희칼럼|성공을 꿈꾸는 사람들의 경제 뉴스|머니 및 금지|및 금지|"
                          r"네이버 모바일에서|네이버에서 헤럴드경제|채널 구독하기|헤럴드 리얼라이프 헤럴드경제 사이트 바로가기 헤럴드경제 무단 전재 및 금지|"
                          r"전자신문|바로가기|전자신문인터넷|인터넷 및 금지|중앙일보|친구추가|구독 1위|"
                          r"한국경제|재배포|기사제보|보도자료|한경닷컴|네이버에서|자세히보기|자세히 보기|모바일한경|구독신청|"
                          r"머니투데이|돈이 보이는 리얼타임 뉴스|연예뉴스|파이낸셜뉴스|무단 전재 금지|"
                          r"C|Users|brsta|brstar|txt") # 높은 가중치를 갖는 단어를 하드코딩 데이터로 제거
    patternList["ElimRecWord"] = pattern7
    pattern8 = re.compile(r'[0-9]') # 숫자만 제거
    patternList["ElimNumOnly"] = pattern8
    pattern9 = re.compile(r"\s{1,}") # 한칸짜리 공백제거
    patternList["Whitespace_one"] = pattern9
    
    return patternList

In [5]:
from os import listdir

# FileList = getFileList("C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts/")
FileList = getFileList("./0314_DownloadedNewstxts")

NewsContent = list() 

for i in range(len(FileList)):
    NewsContent.append(getContent(FileList[i])) # len(NewsContent) = 180


In [6]:
patternList = getPatternList()
for _ in ["Korean", "Whitespace", "Punctuation", "ElimLongEng", "Email", "Domain", "ElimRecWord"]: 
    for i in range(len(FileList)):
        NewsContent[i] = patternList[_].sub(" ", NewsContent[i]) 
print(NewsContent[0]) # 불용어 처리가 모두 끝난 뉴스기사 목록


   오히려 단체측 무선이어폰 연구는 존재치도 않아 국내 참여 연구자도 어리둥절 이엠에프사이언티스에서 일부 언론 보도를 부정하며 직접 보내온 이메일 내용이다 이엠에프사이언티스트 관계자인 조엘 모스코위츠 미국 버클리 캘리포니아대 가정사회건강연구소 소장은 무선 이어폰의 건강 유해성에 대한 보도를 부정했다 고재원   1212 18일 오전 전세계 과학자들이 애플 에어팟과 같은 무선 이어폰이 암 발생 위험을 키울 수 있다는 호소문을 국제연합 과 세계보건기구 에 제출했다는 일부 국내외 언론의 보도가 나오면서 불안감이 확산되고 있다 하지만 실제 이 단체와 호소문에 이름을 올린 과학자들에게 확인한 결과 호소문은 4년전 제출됐던 것이며 또 특정 제품이나 제조사를 언급하지 않은 것으로 확인됐다 18일 데일리메일과   등 국내외 일부 언론에 따르면 전 세계 과학자 247명이 무선 이어폰의 비이온화 전자기장 이 암을 유발할 위험 우려가 있다며 과 에 호소문을 제출했다고 전했다 이들 매체들은 호소문에 애플 에어팟이 에 관한 법적 기준치를 준수하고 있지만 장시간 노출될 경우 건강에 좋지 않을 수 있다 는 내용이 포함됐다고 보도했다 이들 매체들은 이 호소문에는 전 세계 42개국 과학자 247명이 서명을 했다고 전했다 여기에는 한국의 연세대 한양대 가톨릭대 단국대 중앙대 경북대 한림대 소속 과학자 15명의 이름도 포함됐다 하지만 취재 결과 호소문 작성을 주도한 비영리단체 이엠에프사이언티스트 는 애플 에어팟과 같은 무선 이어폰에 대한 유해성을 주장하지 않은 것으로 확인됐다 동아사이언스가 이엠에프사이언티스트에 직접 이메일로 확인한 결과 일부 언론 보도가 호소문에 대한 부정확한 내용을 담고 있다 며 무선 블루투스의 자기장에 머리가 장시간 노출될 시의 안정성에 대한 연구는 존재하지 않는다 고 밝혔다 이엠에프사이언티스트에 따르면 이 단체는 지난 2015년 5월 실제로 전세계 과학자 190명의 서명을 받아 과 유엔환경계획 에 국제 과학자 호소문 을 제출하기는 했다 당시 호소문에는 전세계 저명한 학술

In [7]:
from nltk.tokenize import sent_tokenize, word_tokenize
from konlpy.tag import Kkma
ma = Kkma()

# 단일 뉴스 기사를 토큰화해주는 함수
def Tokenizer(NewsContent):
    dictTerm = list()
    dictPos = list()
    dictNoun = list()
    dictNgram = list()
    th = 1 
    
    for sentence in sent_tokenize(NewsContent):
        for token in word_tokenize(sentence):
            if len(token) > th:
                dictTerm.append(token)
                # 아래서부터는 list이기때문에 extend를 사용해야 함. 
                dictPos.extend([morpheme for morpheme in ma.morphs(token) if len(morpheme) > th]) # 형태소 분석결과를 extend
                dictNoun.extend([noun for noun in ma.nouns(token) if len(noun) > th]) # 명사 단위로 잘라 extend
                dictNgram.extend(ngramUmjeol(token)) # 바이그램을 리턴
                
    # 빠른 속도 및 중복 데이터 처리를 위해 set사용 (유일한 단어만 남김)
    dictTerm = list(set(dictTerm))
    dictPos = list(set(dictPos))
    dictNoun = list(set(dictNoun))
    dictNgram = list(set(dictNgram))
        
    return dictTerm, dictPos, dictNoun, dictNgram

In [8]:
# TokenizedNewsContent = list()
# 
# for i in range(len(FileList)):
#     TokenizedNewsContent.append(Tokenizer(NewsContent[i]))

In [9]:
# # len(dictTerm), len(dictPos), len(dictNoun), len(dictNgram) of 0th txt file
# print(len(TokenizedNewsContent[0][0]), len(TokenizedNewsContent[0][1]), len(TokenizedNewsContent[0][2]), len(TokenizedNewsContent[0][3]))
# 
# # dictTerm, dictPos, dictNoun, dictNgram 테스트용
# # for i in range(len(FileList)):
# #     print(str(i) + "th txt : " + str(len(list(set(TokenizedNewsContent[i][0] + TokenizedNewsContent[i][1] + TokenizedNewsContent[i][2] + TokenizedNewsContent[i][3])))))

In [10]:
# # 0 : dictTerm, 1 : dictPos, 2 : dictNoun, 3 : dictNgram
# print(TokenizedNewsContent[0][0])

## 2. TDF-ID 방식으로 검색 수행해보기
WIP(2019-03-29 10:28 PM) 



In [11]:
collection = [] 
for i in range(len(NewsContent)):
    collection.append((str(FileList[i]), NewsContent[i])) # (파일경로, 불용어 처리 완료된 기사) 튜플 구성 
print(collection[0])

# in-memory (Hash Key 값)
# 전체 색인어 목록(Dictionary)
# {단어1:포스팅위치, 단어2:포스팅위치, ...}
queryPosting = dict()

# 전체 문서 목록(Dictionary)
# [0:문서1, 1:문서2, ...]
globalDocument = list()

# disk
# 사전에 있는 색인어 중, 어느 문서에서, 몇 번 나타났는지
# [(단어 idx, 문서 idx, 빈도, 다음주소), ...]
# [0:Tuple(lexiconIdx, documentIdx, freq, 다음포스팅위치-fptr)]
# 메모리 X, File OK
globalPosting = list()

('./0314_DownloadedNewstxts/IT과학-0000003610.txt', '   오히려 단체측 무선이어폰 연구는 존재치도 않아 국내 참여 연구자도 어리둥절 이엠에프사이언티스에서 일부 언론 보도를 부정하며 직접 보내온 이메일 내용이다 이엠에프사이언티스트 관계자인 조엘 모스코위츠 미국 버클리 캘리포니아대 가정사회건강연구소 소장은 무선 이어폰의 건강 유해성에 대한 보도를 부정했다 고재원   1212 18일 오전 전세계 과학자들이 애플 에어팟과 같은 무선 이어폰이 암 발생 위험을 키울 수 있다는 호소문을 국제연합 과 세계보건기구 에 제출했다는 일부 국내외 언론의 보도가 나오면서 불안감이 확산되고 있다 하지만 실제 이 단체와 호소문에 이름을 올린 과학자들에게 확인한 결과 호소문은 4년전 제출됐던 것이며 또 특정 제품이나 제조사를 언급하지 않은 것으로 확인됐다 18일 데일리메일과   등 국내외 일부 언론에 따르면 전 세계 과학자 247명이 무선 이어폰의 비이온화 전자기장 이 암을 유발할 위험 우려가 있다며 과 에 호소문을 제출했다고 전했다 이들 매체들은 호소문에 애플 에어팟이 에 관한 법적 기준치를 준수하고 있지만 장시간 노출될 경우 건강에 좋지 않을 수 있다 는 내용이 포함됐다고 보도했다 이들 매체들은 이 호소문에는 전 세계 42개국 과학자 247명이 서명을 했다고 전했다 여기에는 한국의 연세대 한양대 가톨릭대 단국대 중앙대 경북대 한림대 소속 과학자 15명의 이름도 포함됐다 하지만 취재 결과 호소문 작성을 주도한 비영리단체 이엠에프사이언티스트 는 애플 에어팟과 같은 무선 이어폰에 대한 유해성을 주장하지 않은 것으로 확인됐다 동아사이언스가 이엠에프사이언티스트에 직접 이메일로 확인한 결과 일부 언론 보도가 호소문에 대한 부정확한 내용을 담고 있다 며 무선 블루투스의 자기장에 머리가 장시간 노출될 시의 안정성에 대한 연구는 존재하지 않는다 고 밝혔다 이엠에프사이언티스트에 따르면 이 단체는 지난 2015년 5월 실제로 전세계 과학자 190명의 서명을 받아 과

In [12]:
for (docName, docContent) in collection:
    # Pointer 대체용, Key, Document 이름은 절대로 겹치지 않는다는 가정
    docIdx = len(globalDocument)
    globalDocument.append(docName)
    
    # {단어idx:빈도, 단어idx:빈도, ...}
    localPosting = dict()
    
    # Local 작업
    for term in docContent.lower().split():
        # Local에 대해서, 없으면 추가
        if term not in localPosting.keys():
            localPosting[term] = 1
        # 있으면, 빈도 증가
        else:
            localPosting[term] += 1
     
    # Global Marge
    # fp -> struct(단어, 빈도) (localPosting)
    for indexTerm, termFreq in localPosting.items():
        if indexTerm not in queryPosting.keys(): 
            lexiconIdx = len(queryPosting)
            postingIdx = len(globalPosting) # fseek
            postingData = (lexiconIdx, docIdx, termFreq, -1)
            globalPosting.append(postingData)
            queryPosting[indexTerm] = postingIdx # globalPosting 위치(ptr:idx)
        else: # 기존 단어의 idx 가져오기
            lexiconIdx = list(queryPosting.keys()).index(indexTerm)
            postingIdx = len(globalPosting)
            beforeIdx = queryPosting[indexTerm]
            postingData = (lexiconIdx, docIdx, termFreq, beforeIdx)
            globalPosting.append(postingData)
            queryPosting[indexTerm] = postingIdx
            
#     print(localPosting)
# print(globalDocument)

#         if term not in globalLexicon.keys():
#             lexiconIdx = len(globalLexicon) 0

In [13]:
# queryPosting, globalDocument

In [14]:
# globalPosting

In [15]:
for indexTerm, postingIdx in queryPosting.items():
    # indexTerm:단어: postingIdx:위치, ...
    # print(indexTerm)
    
    while True: # Posting Next:-1
        if postingIdx == -1:
            break
            
        postingData = globalPosting[postingIdx]
        # print('  DocName:{0} - TermFreq:{1} - Next:{2}'.format(globalDocument[postingData[1]], postingData[2], postingData[3]))
        postingIdx = postingData[3]

In [16]:
globalPosting[queryPosting['이어폰']]

(775, 2, 1, -1)

In [17]:
globalPosting[globalPosting[queryPosting['이어폰']][3]]   # 다음 주소가 "-1" 일때 까지 반복해서 찾음

(1023, 59, 1, 18851)

## 3. TF-IDF 방식과 cosine simmilarity으로 검색해보기

In [18]:
from math import log10

# TF 구하기
def binaryTF(freq):
    if freq > 0:
        return 1
    else:
        return 0

def rawTF(freq):
    return freq

def basicTF(freq, totalFreq):
    return freq/totalFreq

def logTF(freq):
    if freq > 0:
        return 1+log10(freq)
    else:
        return 0

def doubleNormalTF(K, freq, maxFreq): 
    return K + ((1-K) * (freq/maxFreq))

In [19]:
# IDF(Inverse Document Frequency 구하기
def unaryIDF():
    return 1

def basicIDF(N, df):
    return log10(N/df)

def smoothigIDF(N, df):
    return log10((N+1)/df)

def probabilityIDF(N, df):
    return log10((N-df+1)/df)

In [20]:
def innerProduct(x, y): # 내적 연산
    return x * y

In [21]:
globalLexicon = dict()
globalDocument = list()
globalPosting = list()

for (docName, docContent) in collection:
    docIdx = len(globalDocument)
    globalDocument.append(docName)
    
    localPosting = dict()
    
    for term in docContent.lower().split():
        if term not in localPosting.keys():
            localPosting[term] = 1
        else:
            localPosting[term] += 1
    
    maxFreq = max(localPosting.values())
    
    for indexTerm, termFreq in localPosting.items():
        if indexTerm not in globalLexicon.keys():
            lexiconIdx = len(globalLexicon)
            postingIdx = len(globalPosting)
            postingData = [lexiconIdx, docIdx, doubleNormalTF(0, termFreq, maxFreq), -1]
            globalPosting.append(postingData)
            globalLexicon[indexTerm] = postingIdx
        else:
            lexiconIdx = list(globalLexicon.keys()).index(indexTerm)
            postingIdx = len(globalPosting)
            beforeIdx = globalLexicon[indexTerm]
            postingData = [lexiconIdx, docIdx, doubleNormalTF(0, termFreq, maxFreq), beforeIdx]
            globalPosting.append(postingData)
            globalLexicon[indexTerm] = postingIdx

In [22]:
# globalPosting

In [23]:
N = len(globalDocument)
globalLexiconIDF = dict()
globalDocumentLength = dict()

for indexTerm, postingIdx in globalLexicon.items():
    # indexTerm : 단어, postingIdx :위치(포인터역할), ...
    df = 0
    oldPostingIdx = postingIdx
    
    while True: # ptr Next: -1 
        if postingIdx == -1:
            break
        
        df += 1
        postingData = globalPosting[postingIdx]
        postingIdx = postingData[3]
        
    idf = smoothigIDF(N, df)
    globalLexiconIDF[indexTerm] = idf # 나중에 단어가 나왔을때 idf를 불러올 수 있도록 하기 위함
    postingIdx = oldPostingIdx
    # print('단어 : {0} / IDF : {1}'.format(indexTerm, idf))
    
    while True:
        if postingIdx == -1:
            break
        
        postingData = globalPosting[postingIdx]
        TF = postingData[2]
        globalPosting[postingIdx][2] *= idf
        # print('    Documents:{0} / TF:{1} / TF-IDF:{2}'.format(globalDocument[postingData[1]], 
        #                                                        TF, globalPosting[postingIdx][2]))
        
        postingIdx = postingData[3]
        
        # 색인하는 과정에서 사실상 검색까지 모두 한 셈이 되므로 색인 과정에만 시간이 들어 굉장히 빠른 연산속도를 보임. 
        # 키 에러 방지용(키값이 있다면 제곱해서 누적해주면 됨)
        if postingData[1] not in globalDocumentLength.keys():
            globalDocumentLength[postingData[1]] = globalPosting[postingIdx][2] ** 2
        else:
            globalDocumentLength[postingData[1]] += globalPosting[postingIdx][2] ** 2

In [24]:
query = '이어폰 과 스마트폰' # Document

queryPosting = dict()

for term in query.lower().split():
    if term not in queryPosting.keys():
        queryPosting[term] = 1
    else:
        queryPosting[term] += 1
        
maxFreq = max(queryPosting.values())

for indexTerm, termFreq in queryPosting.items():
    queryPosting[indexTerm] = doubleNormalTF(0.5, termFreq, maxFreq)

In [25]:
queryPosting

{'이어폰': 1.0, '과': 1.0, '스마트폰': 1.0}

In [26]:
candidateList = dict()

for indexTerm, queryWeight in queryPosting.items(): # keys에 있는지 검사 후 query가 있을 경우에 한해 처리하기 위함
    if indexTerm in globalLexicon.keys():
        postingIdx = globalLexicon[indexTerm]
        
        while True: # ptr Next: -1
            if postingIdx == -1:
                break
            
            postingData = globalPosting[postingIdx]
            postingIdx = postingData[3]
            documentWeight = postingData[2]
            
            
            # 키 에러 방지용
            if postingData[1] not in candidateList.keys():
                candidateList[postingData[1]] = innerProduct(queryWeight, documentWeight)
            else:
                candidateList[postingData[1]] += innerProduct(queryWeight, documentWeight)

for documentIdx, sumProduct in candidateList.items():
    candidateList[documentIdx] /= globalDocumentLength[documentIdx]

In [27]:
# 정렬 (reversed = False는 작은값부터 큰 값 순서)
resultList = sorted(candidateList.items(), key=lambda x:x[1], reverse=True)

print('query: ', query)

for i, (documentIdx, distance) in enumerate(resultList):
    print('순위:{0} / 문서:{1} / 유사도:{2}'.format((i+1), globalDocument[documentIdx], distance))
    print('   document:{0}'.format(collection[documentIdx][1]))

query:  이어폰 과 스마트폰
순위:1 / 문서:./0314_DownloadedNewstxts/생활문화-0000749396.txt / 유사도:0.07325784520145984
   document:   불과 몇 년 전 만해도 오십견은 말 뜻 그대로 50세 이상의 중 장년층 들에게서 주로 발생하는 질환이었다 하지만 요즘은 삼십견 사십견 이라는 신조어까지 생길 정도로 장시간 컴퓨터 및 스마트폰 사용 무리한 운동 스포츠 활동의 증가 등 생활 패턴의 변화로 30 40 대에서도 안심할 수 없는 질환이 되었다 주로 50세 전후에 나타나는 증상이라 하여 쉽게 말해 오십견 이라고도 명하지만 정확한 의학적 명칭은 유착성 관절낭염 또는 동결견 이라고 하며 어깨 관절을 감싸고 있는 관절낭에 유착이 발생하는 질환이다 오십견 초기에는 통증이 심하지 않아 방치하는 경우가 많은데 점차 통증이 심해지면 어깨 관절의 움직임에 제한이 발생하여 옷을 입거나 머리감기가 매우 불편하며 밤에 통증이 심해 잠을 자기 힘들 지경까지 이르게 된다 오십견은 주로 단계적 치료를 통해 완치가 가능한 질환으로 처음부터 수술을 진행하지 않는다 오십견은 수압팽창술 과 브리즈망 비관헐적 관절 수동술 을 통해 통증 없이 어깨를 바로 움직일 수 있다 수압팽창술은 섬유화가 진행되어 쪼그라든 관절주머니 안에 초음파를 이용하여 주사로 염증제거 약물을 투여함으로써 염증을 가라앉히고 생리식염수를 주입하여 관절주머니를팽창시킨 후 유착된 부분을 풀어주는 방법이다 환자의 상태에 따라 유착이 심한 경우에는 2 3 차례 점진적으로 시행할 수도 있다 브리즈망 관절수동술은 어깨로 가는 신경을 부분 마취한 후 숙련된 의사가 유착된 어깨관절에 염증치료제와 유착방지제를 주입해 굳어진 어깨관절과 근육을 체계적으로 풀어줘서움직임이 제한되었던 범위를 회복시켜주는 시술로 오랫동안 통증에 시달렸던 만성통증환자들에게 좋은 효과를 기대할 수 있는 시술이다 신경을 부분마취하기 때문에 시술시 통증은 거의 없으며 시술자체는 10분 내외로 소요되어 입원치료없이

## TDM-DTM-TWM

In [28]:
from konlpy.corpus import kobill
from collections import defaultdict
from konlpy.tag import Kkma

ma = Kkma().morphs

In [29]:
print(collection[1][1])

   지난 3월11일 유플러스는 한양대학교 에이스 랩 과 공동 개발한 자율주행차를 공개했습니다 유플러스는  를 통해 세계 최초 5 기반의 자율주행차가 일반 차량들과 서울 도심 도로를 달린다 라고 표현했습니다 어딘가 익숙한데요 2017년 평창동계올림픽 기간 동안 는 강릉 평창 등지에서 5 협력 자율주행 버스를 선보였습니다 이때 의 자율주행 버스에도 세계 최초 타이틀이 붙었습니다 텔레콤은 지난해 세계 최초로 복수의 자율주행차가 5 기반 협력 자율주행에 성공했다고 발표했고요 12월1일 텔레콤은 5 전파를 송출하면서 경기도 화성의 자율주행 테스트베드 시티 와 시흥 일반도로에서 5 자율주행차 테스트 운행에 나섰다고  를 통해 밝히기도 했습니다 이 때문에 유플러스  간담회 말미  들의 질문이 쏟아졌습니다 세계 최초가 확실하냐는 거였죠 유플러스는 세계 최초가 맞다 고 확언했습니다 이통사들은 불편한 심기를 내비쳤습니다 텔레콤 관계자는 이미 5 로 자율주행을 한 사례가 있는데 유플러스가 도심에서 자율주행을 했다고 해서 세계 최초 5 자율주행이라 표현하는 건 무리가 있다 라고 지적했습니다 관계자는 남의 집 잔치에 굳이 끼어들고 싶지는 않다 라고 언급을 피했지만 우리도 5 자율주행을 했는데 환경과 범위를 한정하고 좁혀서 세계 최초라고 말하면 그걸 아니라고 할 수는 없다 라는 입장을 밝혔습니다 탐나는 수식어 세계 최초 그래서 누가 세계 최초 5 자율주행을 한 거냐고요 기준에 따라 다릅니다 예를 들어 텔레콤이 지난해 세계 최초라고 했던 건 5 자율주행차가 서로의 운행 경로를 공유하면서 협력 운행에 성공한 것은 이번이 세계 처음 이라던 거였는데요 유플러스의 자율주행은 5 상용 기지국과 연결된 상태로 도심도로에서 자율주행한 세계 최초의 사례라는 겁니다 강종오 유플러스 부문 미래기술담당은 5 통신기술을 상용화해 실제 도심에서 자율주행에 이용한 것은 이번이 세계 최초 라며 유플러스는 통신사 중 5 를 가장 빠르고 촘촘하게 준비하고 있다 라고 말했습니다 참고기사 차 2대로 세계 첫 5 자율주

In [30]:
def getDocReprByDefaultDict():
    docRepr = defaultdict(lambda: defaultdict(int))
    
    count = 0
    for docName in collection:
        print(str(count) + "th document has been analyzed.")
        for token in (docName[1]).split(): 
            for morpheme in ma(token):
                docRepr[docName[0]][morpheme] += 1
        count +=1
    return docRepr

In [31]:
DTM = getDocReprByDefaultDict()

0th document has been analyzed.
1th document has been analyzed.
2th document has been analyzed.
3th document has been analyzed.
4th document has been analyzed.
5th document has been analyzed.
6th document has been analyzed.
7th document has been analyzed.
8th document has been analyzed.
9th document has been analyzed.
10th document has been analyzed.
11th document has been analyzed.
12th document has been analyzed.
13th document has been analyzed.
14th document has been analyzed.
15th document has been analyzed.
16th document has been analyzed.
17th document has been analyzed.
18th document has been analyzed.
19th document has been analyzed.
20th document has been analyzed.
21th document has been analyzed.
22th document has been analyzed.
23th document has been analyzed.
24th document has been analyzed.
25th document has been analyzed.
26th document has been analyzed.
27th document has been analyzed.
28th document has been analyzed.
29th document has been analyzed.
30th document has be

In [32]:
DTM

defaultdict(<function __main__.getDocReprByDefaultDict.<locals>.<lambda>()>,
            {'./0314_DownloadedNewstxts/IT과학-0000003610.txt': defaultdict(int,
                         {'오히려': 1,
                          '단체': 5,
                          '측': 1,
                          '무선': 7,
                          '이어폰': 6,
                          '연구': 6,
                          '는': 19,
                          '존재': 3,
                          '하': 34,
                          '지': 10,
                          '도': 7,
                          '않': 7,
                          '아': 3,
                          '국내': 1,
                          '참여': 1,
                          '연구자': 1,
                          '어리둥절': 1,
                          '이': 33,
                          '엠': 6,
                          '에프': 6,
                          '사이': 6,
                          '언': 6,
                          '티스': 4,
                          '에서': 2,
     

In [33]:
for fileName, termList in DTM.items():
    for term, freq in termList.items():
        print(termList.items())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [34]:
def convertInvertedDocument(DTM):
    TDM = defaultdict(lambda: defaultdict(int))
    
    for fileName, termList in DTM.items():  
        maxFreq = max(termList.values())
        for term, freq in termList.items():
            TDM[term][fileName] = doubleNormalTF(0, freq, maxFreq)
            
    return TDM # 빈도와 가중치를 동시에 갖게 된다.

In [35]:
TDM = convertInvertedDocument(DTM) # 0322_main.ipynb와 달리 빈도와 가중치를 동시에 갖게 된다.
print(TDM)

defaultdict(<function convertInvertedDocument.<locals>.<lambda> at 0x000001D511FED048>, {'오히려': defaultdict(<class 'int'>, {'./0314_DownloadedNewstxts/IT과학-0000003610.txt': 0.029411764705882353, './0314_DownloadedNewstxts/경제-0002221228.txt': 0.02, './0314_DownloadedNewstxts/사회-0001181453.txt': 0.018867924528301886, './0314_DownloadedNewstxts/생활문화-0000025147.txt': 0.022727272727272728, './0314_DownloadedNewstxts/세계-0002220514.txt': 0.009523809523809525, './0314_DownloadedNewstxts/정치-0000372172.txt': 0.03571428571428571, './0314_DownloadedNewstxts/정치-0002891787.txt': 0.037037037037037035, './0314_DownloadedNewstxts/정치-0003204554.txt': 0.05}), '단체': defaultdict(<class 'int'>, {'./0314_DownloadedNewstxts/IT과학-0000003610.txt': 0.14705882352941177, './0314_DownloadedNewstxts/사회-0000373989.txt': 0.022222222222222223, './0314_DownloadedNewstxts/사회-0003895110.txt': 0.16666666666666666, './0314_DownloadedNewstxts/세계-0000372166.txt': 0.03636363636363636, './0314_DownloadedNewstxts/정치-0000372172.t

In [36]:
# Term - Document => Term-Weight
N = len(DTM)

def TDM2TWM(TDM): # TDM을 받아 term에 대한 weight가 들어있는 값으로 리턴
    TWM = defaultdict(lambda: defaultdict(float))
    DVL = defaultdict(float)
    
    for term, tfList in TDM.items():
        df = len(tfList)
        idf = basicIDF(N, df)
        
        for fileName, tf in tfList.items():
            # 각 문서에서 각 단어가 차지하는 단어
            TWM[term][fileName] = tf * idf
            DVL[fileName] += TWM[term][fileName] ** 2
        
    return TWM, DVL

In [37]:
TWM, DVL = TDM2TWM(TDM)

In [38]:
TWM # 0322_main.ipynb와 달리 빈도와 가중치를 동시에 갖게 된다.

defaultdict(<function __main__.TDM2TWM.<locals>.<lambda>()>,
            {'오히려': defaultdict(float,
                         {'./0314_DownloadedNewstxts/IT과학-0000003610.txt': 0.025737095982108826,
                          './0314_DownloadedNewstxts/경제-0002221228.txt': 0.017501225267834004,
                          './0314_DownloadedNewstxts/사회-0001181453.txt': 0.016510589875315094,
                          './0314_DownloadedNewstxts/생활문화-0000025147.txt': 0.019887755986175003,
                          './0314_DownloadedNewstxts/세계-0002220514.txt': 0.008333916794206668,
                          './0314_DownloadedNewstxts/정치-0000372172.txt': 0.031252187978275,
                          './0314_DownloadedNewstxts/정치-0002891787.txt': 0.032409676421914814,
                          './0314_DownloadedNewstxts/정치-0003204554.txt': 0.04375306316958501}),
             '단체': defaultdict(float,
                         {'./0314_DownloadedNewstxts/IT과학-0000003610.txt': 0.12868547991054413,
    

In [40]:
DVL

defaultdict(float,
            {'./0314_DownloadedNewstxts/IT과학-0000003610.txt': 2.2480947558571653,
             './0314_DownloadedNewstxts/경제-0002221228.txt': 1.0263448158486843,
             './0314_DownloadedNewstxts/사회-0001181453.txt': 0.5045571585047343,
             './0314_DownloadedNewstxts/생활문화-0000025147.txt': 1.8557788229799506,
             './0314_DownloadedNewstxts/세계-0002220514.txt': 0.40469509594235514,
             './0314_DownloadedNewstxts/정치-0000372172.txt': 1.3231624392051795,
             './0314_DownloadedNewstxts/정치-0002891787.txt': 0.6233704943130739,
             './0314_DownloadedNewstxts/정치-0003204554.txt': 1.3987285583226472,
             './0314_DownloadedNewstxts/사회-0000373989.txt': 1.2735804326251052,
             './0314_DownloadedNewstxts/사회-0003895110.txt': 2.3641576958135735,
             './0314_DownloadedNewstxts/세계-0000372166.txt': 0.8162925228877954,
             './0314_DownloadedNewstxts/정치-0002893284.txt': 1.9740747591839007,
             './

In [41]:
query = '(서울=연합뉴스) 차지연 기자 = 국회 행정안전위원회는 4일 오후 전체회의를 열고 진영 행정안전부 장관 후보자에 대한 인사청문 경과보고서를 채택했다. 진 후보자에 대한 청문보고서 채택은 문재인 정부 2기 내각 장관 후보자 7명 중 낙마한 두 후보자를 제외한 5명 가운데 박양우 문화체육관광부 장관 후보자, 문성혁 해양수산부 장관 후보자에 이어 세 번째다. 행안위는 진 후보자 청문보고서에 적격과 부적격 의견을 모두 담아 채택했다. 행안위는 보고서에서 높은 전문성과 정책적 이해도 등을 고려할 때 충분한 수준의 직무수행 역량을 갖췄으므로 적격이라는 의견이 제시됐다고 밝혔다. 적격 이유로는 지방분권과 지역균형발전, 자치경찰제 도입, 미세먼지 대책 등 현안에 대한 적극적 추진 의지와 철저한 자기 관리, 보건복지부 장관 역임 당시 보여준 소신과 행정경험 등이 거론됐다.'

queryRepr = defaultdict(int)

for token in query.split():
    for morpheme in ma(token):
        queryRepr[morpheme] += 1 # 빈도도 쉽게 구할 수 있다. 
        
maxFreq = max(queryRepr.values())
queryWeight = defaultdict(float)

for token, freq in queryRepr.items():
    if token in TWM.keys():
        tf = doubleNormalTF(0.5, freq, maxFreq)
        df = len(TWM[token])
        idf = basicIDF(N, df)
        queryWeight[token] = tf * idf

In [42]:
queryWeight

defaultdict(float,
            {'서울': 0.21386882346277838,
             '연합': 0.33865874512197885,
             '뉴스': 0.0990513332188207,
             '차지': 0.5625,
             '연': 0.4922219606578313,
             '국회': 0.41442669293929796,
             '행정': 0.8944581220189871,
             '안전': 0.5469132896198126,
             '위원회': 0.52484243083278,
             '는': 0.0,
             '4': 0.2042875701072144,
             '일': 0.04453945090178896,
             '오후': 0.3931706274390106,
             '전체': 0.52484243083278,
             '회의': 0.43771007834079956,
             '를': 0.015315021363917143,
             '열': 0.5625,
             '고': 0.008281831961647338,
             '진영': 1.0002100783407994,
             '부': 0.5625,
             '장관': 0.669425852017259,
             '에': 0.0059306314774682815,
             '대하': 0.15252101536124502,
             'ㄴ': 0.007299238741499423,
             '인사': 0.5625,
             '청문': 1.222478984638755,
             '보고서': 0.88206844

## KNN search
[KNN -> K 선정하기]
9. K개의 가장 유사한 문서들의 카테고리 추출
10. 카테고리별 투표를 통해 가장 유사한 카테고리 선정

In [43]:
from math import sqrt

candidateList = defaultdict(float)

for token, weight in queryWeight.items():
    for fileName, tfidf in TWM[token].items():
        print('{0} : {1} = {2} * {3}'.format(token, fileName, weight, tfidf))
        candidateList[fileName] += innerProduct(weight, tfidf)
    
for fileName, sumProduct in candidateList.items():
    candidateList[fileName] /= sqrt(DVL[fileName])

서울 : ./0314_DownloadedNewstxts/IT과학-0000023756.txt = 0.21386882346277838 * 0.011881601303487688
서울 : ./0314_DownloadedNewstxts/IT과학-0002158229.txt = 0.21386882346277838 * 0.010005558992410684
서울 : ./0314_DownloadedNewstxts/경제-0000168149.txt = 0.21386882346277838 * 0.022365367159506235
서울 : ./0314_DownloadedNewstxts/경제-0000205610.txt = 0.21386882346277838 * 0.010275979505719082
서울 : ./0314_DownloadedNewstxts/경제-0000389333.txt = 0.21386882346277838 * 0.006444258334095018
서울 : ./0314_DownloadedNewstxts/경제-0000391651.txt = 0.21386882346277838 * 0.06709610147851872
서울 : ./0314_DownloadedNewstxts/경제-0001267098.txt = 0.21386882346277838 * 0.0760422483423212
서울 : ./0314_DownloadedNewstxts/경제-0001510681.txt = 0.21386882346277838 * 0.06336854028526767
서울 : ./0314_DownloadedNewstxts/경제-0002221228.txt = 0.21386882346277838 * 0.030416899336928482
서울 : ./0314_DownloadedNewstxts/경제-0002515783.txt = 0.21386882346277838 * 0.03564480391046307
서울 : ./0314_DownloadedNewstxts/사회-0000373989.txt = 0.21386882

In [44]:
# fileName으로부터 카테고리만 남기기

import re
CategoryList = ['C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts//IT과학-0000003610.txt',
              "C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts//세계-0000372983.txt",
              'C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts//경제-0002221228.txt',
              "C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts//생활문화-0000025147.txt",
             "C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts//사회-0001181453.txt",
              "C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts//정치-0000372172.txt"]

patternList = getPatternList()
CategoryResult = []

def ElimPunc(Filename):
    for _ in ["ElimLongEng", "ElimNumOnly", "Punctuation", "ElimRecWord", "Whitespace"]: 
        Filename = patternList[_].sub("", Filename)
    return Filename
    
# for i in range(len(CategoryList)):
#     for _ in ["ElimLongEng", "ElimNumOnly", "Punctuation", "ElimRecWord", "Whitespace"]: 
#         CategoryList[i] = patternList[_].sub("", CategoryList[i])
#     CategoryResult.append(CategoryList[i])
# print(CategoryResult)

In [45]:
print(ElimPunc(CategoryList[3]))

생활문화


In [46]:
from nltk.tokenize import sent_tokenize

resultList = sorted(candidateList.items(), key=lambda x:x[1], reverse=True)
K = 5

for i, (fileName, similarity) in enumerate(resultList):
    if i < K:
        print('Rank:{0} / Name:{1} / Similarity:{2:.4f}'.format((i+1), fileName, similarity))
        print(getContent(fileName))

Rank:1 / Name:./0314_DownloadedNewstxts/정치-0002892355.txt / Similarity:0.7611





// flash 오류를 우회하기 위한 함수 추가
function _flash_removeCallback() {}

	
	 문재인 대통령이 18일 논란이 확산되고 있는 ‘버닝썬’ 클럽과 경찰과의 유착 의혹 사건과 김학의 전 법무부 차관의 ‘별장 성접대 의혹’, 고(故) 장자연 씨 관련 사건에 대한 진상조사를 지시했다.      문재인 대통령이 18일 오후 청와대에서 박상기 법무부 장관과 김부겸 행정안전부 장관으로부터 '장자연·김학의·버닝썬 사건' 관련 보고를 받고 있다. [청와대 제공]       문 대통령은 이날 청와대에서 박상기 법무부, 김부겸 행정안전부 장관으로부터 관련 보고를 받은 뒤 “과거의 일이지만 진실을 밝히고 스스로의 치부를 드러내고 신뢰받는 사정기관으로 거듭나는 일은 검찰과 경찰의 현 지도부가 조직의 명운을 걸고 책임져야 할 일이라는 점을 명심해달라”고 말했다고 김의겸 청와대 대변인이 전했다.      문 대통령은 해당 사건에 대해 “수사 기관이 고의적인 부실수사를 하거나 더 나아가 적극적으로 진실규명을 가로막고 비호ㆍ은폐한 정황이 보인다”고 말했다. 검찰과 경찰이 범죄에 적극적으로 가담했다는 점을 전제한 말이다. 문 대통령은 그러면서 “공통적인 특징은 사회 특권층에서 일어난 일”이라며 이를 권력형 비리로 규정했다. 사실상의 수사 ‘가이드라인’이다.      김학의 전 법무부 차관이 인천지검 검사장 시절 다문화 가정 초청 법률 교육 및 범죄피해자지원센터와 다문화 가족지원센터 간의 업무 협약 체결식 행사에서 인사말을 하고 있다. 중앙포토       문 대통령은 특히 각 사건이 발생한 시기에 대해 “드러난 범죄 행위 시기와 유착관계 시기는 과거 정부 때의 일”이라고 이번 지시가 사실상 과거 정부 수사 기관에서 발생한 일에 대한 진상규명에 맞춰져 있음을 분명히 했다.      그는 “검찰과 경찰이 권력형 

In [47]:
from collections import Counter

# print(resultList)
FinalIRResult = []
for i in range(len(resultList)):
    FinalIRResult.append(ElimPunc(resultList[i][0]))
    #print(ElimPunc(resultList[i][0]))
# print(FinalIRResult)
modList = Counter(FinalIRResult).most_common()

print("-> Given Query : " + query)
print("-> 해당 쿼리의 카테고리는 " + modList[0][0] + " 입니다.")

-> Given Query : (서울=연합뉴스) 차지연 기자 = 국회 행정안전위원회는 4일 오후 전체회의를 열고 진영 행정안전부 장관 후보자에 대한 인사청문 경과보고서를 채택했다. 진 후보자에 대한 청문보고서 채택은 문재인 정부 2기 내각 장관 후보자 7명 중 낙마한 두 후보자를 제외한 5명 가운데 박양우 문화체육관광부 장관 후보자, 문성혁 해양수산부 장관 후보자에 이어 세 번째다. 행안위는 진 후보자 청문보고서에 적격과 부적격 의견을 모두 담아 채택했다. 행안위는 보고서에서 높은 전문성과 정책적 이해도 등을 고려할 때 충분한 수준의 직무수행 역량을 갖췄으므로 적격이라는 의견이 제시됐다고 밝혔다. 적격 이유로는 지방분권과 지역균형발전, 자치경찰제 도입, 미세먼지 대책 등 현안에 대한 적극적 추진 의지와 철저한 자기 관리, 보건복지부 장관 역임 당시 보여준 소신과 행정경험 등이 거론됐다.
-> 해당 쿼리의 카테고리는 정치 입니다.
